In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from easydict import EasyDict
from opt import *
import itertools


def beale_function(x: torch.tensor):
    return (1.5 - x[1] + x[1] * x[2]) ** 2 + (2.25 - x[1] + x[1] * x[2] ** 2) ** 2 + (
            2.625 - x[1] + x[1] * x[2] ** 3) ** 2


for start_point, model in itertools.product(
        [(0, 0, 0), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 1), (0, 3, 1), (0, 3, 3), (0, 3, 5), (0, 3, 2)],
        ["GD", "Adam"]):
    CFG = EasyDict()
    CFG.start_point = start_point
    CFG.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    CFG.model = "GD"
    CFG.project = "opt"
    CFG.lr = 1e-3
    CFG.wandb = True
    CFG.max_iter = int(3 * 1e5)
    CFG.tags = ["v0"]
    if CFG.wandb:
        # from kaggle_secrets import UserSecretsClient
        #
        # user_secrets = UserSecretsClient()
        # secret_value_0 = user_secrets.get_secret("WANDB")
        # wandb.login(key=secret_value_0)
        def wandb_init():
            run = wandb.init(
                project=CFG.project,
                name=f"{CFG.project}-{CFG.model}-{CFG.start_point}",
                config={k: v for k, v in CFG.items() if '__' not in k},
                tags=CFG.tags,
                save_code=True
            )
            return run
    if CFG.wandb:
        run = wandb_init()
    g = eval(CFG.model)(CFG, beale_function, max_iter=CFG.max_iter, lr=CFG.lr)
    g(torch.Tensor(CFG.start_point))
    if CFG.wandb:
        run.finish()

wandb: Currently logged in as: shujakuin. Use `wandb login --relogin` to force relogin


  0%|          | 0/300000 [00:00<?, ?it/s]


TypeError: Optimizer.log() missing 1 required positional argument: 'delta_value'